In [0]:
dbutils.library.installPyPI("mlflow")
dbutils.library.restartPython()
from pyspark.sql.types import StructType
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import *      # for window() function
from typing import List
from pyspark.sql.types import *
import pandas as pd
import time
from datetime import datetime, timedelta, timezone
import mlflow
from pyspark.sql.types import StructType
import dateutil.parser
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.types import StructType

version = "v-18"

In [0]:
%sql
DROP TABLE raw_log_data_delta_PN_test;
DROP TABLE anomalies_data_delta_PN_test;

In [0]:
%sql
CREATE TABLE raw_log_data_delta_PN_test (
  account_id STRING,
  agent_id STRING,
  event STRING,
  timestamp TIMESTAMP
 )
USING DELTA;


CREATE TABLE anomalies_data_delta_PN_test (
  user_id STRING,
  Ips LONG,
  prediction DOUBLE
)
USING DELTA;


In [0]:
from pyspark.sql.types import StructType
#57c7413abca837e974000009
inputPath = "dbfs:/mnt/kafka_raw/57c7413abca837e974000009/"

schema_raw_logs = (  StructType()
  .add("account_id","string")
  .add("agent_id","string")
  .add("event","string")
  .add("timestamp","timestamp") 
)


#reads From S3
eventsDF = (
  spark
    .readStream.queryName("read_S3")
    .schema(schema_raw_logs) # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1) # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)

#writes to Raw table
(eventsDF.writeStream.queryName("write_raw_table")
  .outputMode("append")
  .option("checkpointLocation", "/mnt/delta/events/_checkpoints/etl-from-json_PN_"+version)
  .table("raw_log_data_delta_PN_test")
)




--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1714122276317727> in <module> 
 2 inputPath = "dbfs:/mnt/kafka_raw/testes_PN/" 
 3 
 ----> 4 schema_raw_logs = ( StructType()
 5 . add ( "account_id" , "string" ) 
 6 . add ( "agent_id" , "string" ) 

 NameError : name 'StructType' is not defined

In [0]:
#We may need to user OPTIMIZE, which deals with sm all files, merge them and compact them into larger files
raw_data = spark.readStream.format("delta").table("raw_log_data_delta_PN_test").queryName("read_raw_table")



In [0]:
%sql
select * from raw_log_data_delta_PN_test

In [0]:
fullschema = (  StructType()
  .add("logger_event_id", "string")
  .add("logger_timestamp","timestamp")                
  .add("account_id","string")
  .add("agent_id","string")
  .add("event",StructType())
         .add("actor",StructType()
             .add("user_id","string")
             .add("ip_addresses",ArrayType(StringType()))
             .add("session_id","string")
             .add("impersonated_user_id","string")
             .add("id","string")
             .add("type","string")
             .add("user_agent","string")
         )
         .add("account_id","string")
         .add("event_type","string")
         .add("audit",StructType()
              .add("severity","string")
              .add("resource_id","string")
              .add("operation","string")
              .add("timestamp","timestamp")
              .add("status","string")
         )
         .add("logger_event_id","string")     
         .add("object",StructType())
         .add("timestamp","timestamp") 
   
   .add("timestamp","timestamp") 
)


In [0]:

run_id = "09840597c6e04f279aaa27be313c6e73"
model_uri = "runs:/" + run_id + "/sklearn-model"
model = mlflow.pyfunc.spark_udf(spark, model_uri)


filtered_data = (raw_data
                  .select( "timestamp"    ,from_json("event", fullschema).alias("data"))
                  .withColumn("timestamp",to_timestamp(to_date("timestamp","yyyy-MM-dd")))
                  .select( "timestamp" ,   "data.actor.user_id",  "data.actor.ip_addresses")       
                  .where(col("user_id").isNotNull())
                  #.where(col("timestamp") >= datetime.now().astimezone(timezone.utc).strftime("%Y-%m-%dT00:00:00.000+0000") )
                  .withWatermark("timestamp", "24 hours")
                  .groupBy(col("timestamp"),"user_id").agg(approx_count_distinct('ip_addresses').alias('Ips'))
                  .select("user_id","Ips")
                  .withColumn("prediction", model("Ips"))
                )





In [0]:
#write data to anomalies' table
(filtered_data.writeStream.queryName("write_anomalies_table")
  .outputMode("append")
  .option("checkpointLocation", "/mnt/delta/events/_checkpoints/anomalies_"+version)
  .table("anomalies_data_delta_PN_test")

)
